In [1]:
from xgboost import XGBRanker
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("1000개_질문(0포함).csv")

In [3]:
# llm_rank를 관련성 점수로 변환
# 1위 -> 3점, 2위 -> 2점, 3위 -> 1점, 나머지는 0점 (역순 변환)
def convert_rank_to_score(rank):
    if rank == 1: return 3
    elif rank == 2: return 2
    elif rank == 3: return 1
    else: return 0

# 정답(y)은 정수형으로 변환 (0~10 단계로 변환)
df['relevance_label'] = df['llm_rank'].apply(convert_rank_to_score)

---
### 데이터 분할

In [4]:
# 1. 사용자 유형별로 질문(Group) 리스트 추출
queries_0 = df[df['사용자 유형'] == 0]['질문'].unique() # 입문자 질문들
queries_1 = df[df['사용자 유형'] == 1]['질문'].unique() # 기존 팬 질문들

# 2. 각 유형 내에서 8:2로 학습/테스트 질문 분리
# (전체 데이터에서도 8:2 비율이 유지되면서 테스트셋 구성됨)
train_q0, test_q0 = train_test_split(queries_0, test_size=0.2, random_state=42)
train_q1, test_q1 = train_test_split(queries_1, test_size=0.2, random_state=42)

# 3. 최종 학습 및 테스트 데이터 생성
train_df = df[df['질문'].isin(list(train_q0) + list(train_q1))].sort_values('질문')
test_df = df[df['질문'].isin(list(test_q0) + list(test_q1))].sort_values('질문')

# 4. 피처와 정답 정의 (llm_rank 기반 관련성 점수 활용)
features = ['사용자 유형', 'sbert_score', 'n2v_score', 'vector_score']
X_train = train_df[features]
y_train = train_df['llm_rank'].apply(lambda x: 4-x if x <= 3 else 0) # 1위->3점, 2위->2점...

X_test = test_df[features]
y_test = test_df['llm_rank'].apply(lambda x: 4-x if x <= 3 else 0)

# 5. 그룹 정보 계산
group_train = train_df.groupby('질문').size().to_list()
group_test = test_df.groupby('질문').size().to_list()

---
### 모델 학습

In [ ]:
model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg@3",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [6]:
model.fit(X_train, y_train, group=group_train)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg@3'


🔍 LightGBM 로그 분석 결과
1. `No further splits with positive gain` (가장 많이 뜬 경고)
    - 의미: 나무를 더 잘게 쪼개서 학습하려고 했는데, 더 이상 쪼개봤자 성능 향상에 도움이 안 된다고 판단해서 멈춘 거야.

    - 원인: 우리 데이터(약 1,100행)가 머신러닝 기준으로는 아주 큰 편은 아니고, 피처(4개)가 명확해서 모델이 금방 정답 패턴을 찾았기 때문이야.

    - 결과: 이건 에러가 아니라 모델이 과적합(Overfitting)되지 않게 스스로 공부를 적당히 멈춘 똑똑한 상태라고 보면 돼!

2. `Found whitespace in feature_names`
    - 의미: 피처 이름 중에 '사용자 유형'처럼 공백(띄어쓰기)이 있는 게 있다는 뜻이야.

    - 결과: LightGBM이 알아서 사용자_유형처럼 언더바(_)로 바꿔서 처리했으니 아무 문제 없어.

3. `Found 'eval_at' in params`
    - 의미: eval_at 파라미터가 중복으로 설정됐다는 경고야.

    - 결과: 모델 생성할 때 넣은 설정값이 우선 적용됐다는 뜻이니 이것도 안심해도 돼.

In [7]:
scores = model.predict(X_test)

---
### 모델 검증 $\rightarrow$ 실제 추천 리스트로 변환

In [8]:
# 모델이 뱉은 점수를 test_df에 다시 붙여서, 질문별로 어떤 팀이 1등인지 눈으로 확인

# 1. 테스트 데이터 복사본에 예측 점수 추가
test_results = test_df.copy()
test_results['predicted_score'] = scores

# 2. 질문별로 점수가 높은 순(내림차순)으로 정렬
# '질문'과 'predicted_score' 순으로 정렬
test_results = test_results.sort_values(by=['질문', 'predicted_score'], ascending=[True, False])

# 3. 각 질문에서 모델이 뽑은 Top 3 팀 확인
final_top3 = test_results.groupby('질문').head(3)
print(final_top3[['질문', '매칭팀', 'predicted_score', 'llm_rank']])

# '사용자 유형'별로 필터링해서 저장해두면 나중에 피그마 UI에 넣기 좋아!
final_top3.to_csv("final_recommendation_results.csv", index=False, encoding="utf-8-sig")

                                                     질문            매칭팀  \
1191                   나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.      전북 현대 모터스   
1188                   나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.        NC 다이노스   
1193                   나는 EPL는 처음인데, 낭만 성향이 강한 팀을 추천해줘.      대전 하나 시티즌   
457                    나는 EPL는 처음인데, 명문 성향이 강한 팀을 추천해줘.        NC 다이노스   
459                    나는 EPL는 처음인데, 명문 성향이 강한 팀을 추천해줘.      전북 현대 모터스   
...                                                 ...            ...   
75    나는 키움 히어로즈의 '선수 유출' 같은 스타일이 마음에 들어. 이런 느낌을 가진 ...      뉴캐슬 유나이티드   
77    나는 키움 히어로즈의 '선수 유출' 같은 스타일이 마음에 들어. 이런 느낌을 가진 ...  브라이튼 앤 호브 알비온   
330   나는 페라리의 '고압적인 환경' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른...            리버풀   
328   나는 페라리의 '고압적인 환경' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른...      전북 현대 모터스   
329   나는 페라리의 '고압적인 환경' 같은 스타일이 마음에 들어. 이런 느낌을 가진 다른...           강원FC   

      predicted_score  llm_rank  
1191         1.280450         1  
1188         0.584817         1  
1193     

### NDCG
: 추천된 팀들의 **순서(Ranking)**가 얼마나 정확한지 측정하는 지표.
</br> 1등 팀을 1등으로 맞추는 게 2등으로 맞추는 것보다 더 높은 점수를 받음.

- 0.7 이상: 모델이 꽤 영리하게 추천을 잘해줌

- 0.9 이상: LLM이 매긴 순위랑 거의 똑같이 맞추고 있음

- 만약 점수가 너무 낮게(0.5 이하) 나온다면?
</br>: 모델이 아직 '사용자 유형'이나 '벡터 스코어'의 특징을 제대로 못 잡았다는 신호
</br>$\Rightarrow$ learning_rate를 조절하거나 피처를 더 추가하는 파라미터 튜닝 단계 진행

In [9]:
# 모델 성능 평가하기 (NDCG 지표 확인)

from sklearn.metrics import ndcg_score
import numpy as np

# 1. 질문별 NDCG 점수를 저장할 리스트
ndcg_list = []

# 2. 질문(그룹)별로 루프를 돌며 계산
for qid in test_results['질문'].unique():
    # 해당 질문에 속한 데이터만 추출
    group_data = test_results[test_results['질문'] == qid]
    
    # 실제 정답(llm_rank 기반 점수)과 모델의 예측 점수 추출
    # ndcg_score는 2차원 배열 형식을 원하므로 reshape(1, -1)을 해줘.
    y_true = np.array([group_data['relevance_label']]).reshape(1, -1)
    y_score = np.array([group_data['predicted_score']]).reshape(1, -1)
    
    # NDCG@3 계산 (상위 3개 팀의 순위가 얼마나 정확한가?)
    # 데이터가 3개보다 적을 경우를 대비해 k값을 조절해.
    k = min(len(group_data), 3)
    score = ndcg_score(y_true, y_score, k=k)
    ndcg_list.append(score)

# 3. 최종 평균 NDCG 출력
avg_ndcg = np.mean(ndcg_list)
print(f" 최종 모델 성능 (Average NDCG@3): {avg_ndcg:.4f}")


 최종 모델 성능 (Average NDCG@3): 0.6945


In [10]:
from itertools import product
from xgboost import XGBRanker
from sklearn.metrics import ndcg_score
import numpy as np

param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [4, 6],
    "n_estimators": [200, 400],
    "subsample": [0.8, 1.0]
}

best_score = -1
best_params = None

for lr, md, ne, ss in product(
    param_grid["learning_rate"],
    param_grid["max_depth"],
    param_grid["n_estimators"],
    param_grid["subsample"]
):
    model = XGBRanker(
        objective="rank:ndcg",
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=ss,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        group = group_train
    )

    preds = model.predict(X_test)

    score = ndcg_score(
        [y_test],
        [preds]
    )

    if score > best_score:
        best_score = score
        best_params = {
            "learning_rate": lr,
            "max_depth": md,
            "n_estimators": ne,
            "subsample": ss
        }

print("Best NDCG:", best_score)
print("Best Params:", best_params)


Best NDCG: 0.9232078543117979
Best Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.8}
